In [1]:
import pyDOE2 as ff
import numpy as np
import pandas as pd
from scipy.stats import f as Fisher
from scipy.stats import t as Student

In [ ]:
Authors = ["Danis","Myrat","Alia"]
Author = Authors[2]
Author

'Alia'

In [4]:
factor_count = 2  # Количество факторов
y_count = 3  # Количество опытов для каждой комбинации факторов
N0 = factor_count**2
experiment_count = N0 + 2 * factor_count + 1  # Количество экспериментов
name_y = 'y average'  # Название столбца для построения регрессии
alpha = int(np.sqrt((np.sqrt(N0 * experiment_count) - N0)/2))
betta = (N0 + 2*alpha**2)/experiment_count
print(f"α = {alpha} ; β = {betta}")
table = ff.ccdesign(factor_count, center=(0, 1))  # Матрица планирования
table = np.array(table, dtype=np.int8)
# table


α = 1 ; β = 0.6666666666666666


In [5]:
database = pd.read_excel('Data_' + Author + '.xlsx') 
database = database[database.columns[1:]]
database = database.to_numpy()
#print(database)
data = []
for i in range(y_count):
    data.append(list(zip(np.log(database[y_count*i,:] - 0.43),
                         np.log(database[y_count*i + 1,:]- 0.43),
                         np.log(database[y_count*i + 2,:]- 0.43)))) # хранение опытов для каждой комбинации факторов
    # data.append(list(zip(database[y_count*i,:],
    #                      database[y_count*i + 1,:],
    #                      database[y_count*i + 2,:]))) # хранение опытов для каждой комбинации факторов
# data


In [6]:
dtypes = np.dtype(
    [   
        ("x0", int),
        ("x1", int),
        ("x2", int),
        ("x1x2", int),
        ("(x1)^2", int),
        ("(x2)^2", int),
        ("(x1~)^2", np.float64),
        ("(x2~)^2", np.float64),
        ("y1", np.float64),
        ("y2", np.float64),
        ('y3', np.float64),
        ('y average', np.float64)
    ]
)
df = pd.DataFrame(np.zeros(experiment_count, dtype=dtypes))

In [7]:
for i in range(experiment_count):
    idx1 = table[i, 0]  # x1
    idx2 = table[i, 1]  # x2
    # совокупность значений y
    ys = np.array(data[idx1 + 1][idx2 + 1], dtype=np.float64)
    # print(ys)
    df.iloc[i] = {'x0': 1, 'x1': idx1, 'x2': idx2, 'x1x2': idx1*idx2, '(x1)^2': idx1**2, '(x2)^2': idx2**2,
                '(x1~)^2': idx1**2 - betta, '(x2~)^2': idx2**2 - betta,
                'y1': ys[0], 'y2': ys[1], 'y3': ys[2], 'y average': np.mean(ys)}
# y1 - столбец целевой переменной для построения регрессии
fixed_y = df[name_y]
# df


In [8]:

a = []
# Линейные коэффициенты
tmp = np.zeros(factor_count + 1)
for i,name in enumerate(df.columns[:factor_count + 1]):
    tmp[i] = (1./(N0 + 2*alpha**2)) * np.sum(df[name] * fixed_y)
tmp[0] = (1./experiment_count) * np.sum(fixed_y)
a.append(tmp)
del tmp
# Квадратичные коэффициенты
tmp = np.zeros(factor_count)
for i,name in enumerate(df.columns[(factor_count + 1)*2:(factor_count + 1)*2 + factor_count]):
    tmp[i] = (1./(2*alpha**4)) * np.sum(df[name] * fixed_y)
a.append(tmp)
del tmp
# Парный коэффициент
a.append((1./N0) * np.sum(df['x1x2'] * fixed_y))
print(f"Линейные коэффициенты: {a[0]}\nКвадратичные коэффициенты: {a[1]}\nПарный коэффициент: {a[2]}")
regression = lambda x: np.sum(x*a[0]) + np.sum((x[1:]**2)*a[1]) + x[1]*x[2]*a[2]   #где x[0]=1 - фиктивный фактор

Линейные коэффициенты: [5.60525886 0.172094   0.63259956]
Квадратичные коэффициенты: [-0.03420661 -0.23307517]
Парный коэффициент: 0.003937414196580669


In [9]:
dtypes = np.dtype(
    [
        ('(y1-y_avg)^2', np.float64),
        ('(y2-y_avg)^2', np.float64),
        ('(y3-y_avg)^2', np.float64),
        ("Variance", np.float64),
        ('y regression_' + name_y, np.float64),
        ('Δy^2', np.float64)
    ]
)
df_tmp = pd.DataFrame(np.zeros(experiment_count, dtype=dtypes))
df = pd.concat([df, df_tmp], axis=1)
# df

In [10]:
# df['y regression_' + name_y] = [regression(i) for i in df[df.columns[:factor_count + 1]].to_numpy()]
# df['Δy^2'] = (fixed_y - df['y regression_' + name_y]) ** 2

tmp = df.columns[factor_count*3 + 2:(factor_count*3 + 2) + y_count]

for idx,name in enumerate(df_tmp.columns[:y_count]):
    df[name] = (df[tmp[idx]] - df['y average'])**2
    df['Variance'] += df[name]
del tmp

# df

### Проверка адекватности модели ###

In [11]:
f = experiment_count - (factor_count + 1)
print(f"f = {f}")

S_y = df['Variance'].sum() * (1/(experiment_count * (y_count - 1)))
print(f"S_y = {S_y}")


f = 6
S_y = 0.00032130889423891124


In [12]:
p = 0.01 # Принятый уровень значимости

#### Критерий Стьюдента ####

In [14]:
S_a0 = np.sqrt(np.abs(S_y/(y_count*(experiment_count+2*(alpha**4)))))
S_ai = np.sqrt(np.abs(S_y/(y_count*(N0+2*(alpha**2)))))
S_ail = np.sqrt(np.abs(S_y/(y_count*N0)))
S_aii = np.sqrt(np.abs(S_y/(y_count*(2*(alpha**4)))))
print(f"S_a0 = {S_a0}\nS_ai = {S_ai}\nS_ail = {S_ail}\nS_aii = {S_aii}")

t_table = Student.ppf(1-p/2, experiment_count*(y_count-1)) # Табличный критерий Стьюдента
print(f"Табличное значение критерия Стьюдента: {t_table}")

t = np.zeros(6)
# Проверка линейных коэффициентов
t[0] = np.abs(a[0][0])/S_a0
for i in range(1, len(a[0])):
    t[i] = np.abs(a[0][i])/S_ai
    if t[i] > t_table:
        print(f"Коэффициент a{i} является значимым!")
    else:
        print(f"Коэффициент a{i} является незначимым!")
        a[0][i] = 0
# Проверка квадратичных коэффициентов
for i in range(len(a[1])):
    t[i+len(a[0])] = np.abs(a[1][i])/S_aii
    if t[i+len(a[0])] > t_table:
        print(f"Коэффициент a{i+1}{i+1} является значимым!")
    else:
        print(f"Коэффициент a{i+1}{i+1} является незначимым!")
        a[1][i] = 0
# Проверка парного коэффициента:
t[-1] = np.abs(a[-1])/S_ail
if t[-1] > t_table:
    print(f"Коэффициент a12 является значимым!")
else:
    print(f"Коэффициент a12 является незначимым!")
    a[-1] = 0

S_a0 = 0.0031203578574828376
S_ai = 0.004224984511733026
S_ail = 0.005174528112453921
S_aii = 0.007317887835513187
Табличное значение критерия Стьюдента: 2.878440472713585
Коэффициент a1 является значимым!
Коэффициент a2 является значимым!
Коэффициент a11 является значимым!
Коэффициент a22 является значимым!
Коэффициент a12 является незначимым!


#### Критерий Фишера ####

In [15]:
df['y regression_' + name_y] = [regression(i) for i in df[df.columns[:factor_count + 1]].to_numpy()]
df['Δy^2'] = (fixed_y - df['y regression_' + name_y]) ** 2

S_ad = (1./f) * df['Δy^2'].sum()
print(f"S_ад = {S_ad}")

S_ад = 0.04765804069542732


In [16]:
F_table = Fisher.ppf(1-p, f, experiment_count*(y_count-1)) # Табличный критерий Фишера
print(f"Табличное значение критерия Фишера: {F_table}")

F = S_ad/S_y
print(f"Модель адекватна c F = {F}!") if F < F_table else print(f"Модель неадекватна c F = {F}!")


Табличное значение критерия Фишера: 4.014636507354758
Модель неадекватна c F = 148.32468552828445!


In [17]:
df

,x0,x1,x2,x1x2,(x1)^2,(x2)^2,(x1~)^2,(x2~)^2,y1,y2,y3,y average,(y1-y_avg)^2,(y2-y_avg)^2,(y3-y_avg)^2,Variance,y regression_y average,Δy^2
0,1,-1,-1,1,1,1,0.333333,0.333333,4.716443,4.717337,4.720907,4.718229,0.000003,7.950143e-07,0.000007,0.000011,4.533284,0.034205
1,1,1,-1,-1,1,1,0.333333,0.333333,5.051585,5.064302,5.028279,5.048055,0.000012,2.639635e-04,0.000391,0.000668,4.877472,0.029099
2,1,-1,1,-1,1,1,0.333333,0.333333,5.967095,5.972715,5.988385,5.976065,0.000080,1.122269e-05,0.000152,0.000243,5.798483,0.031535
3,1,1,1,1,1,1,0.333333,0.333333,6.338541,6.321074,6.305308,6.321641,0.000286,3.213881e-07,0.000267,0.000553,6.142671,0.032030
4,1,-1,0,0,1,0,0.333333,-0.666667,5.554780,5.595715,5.562488,5.570994,0.000263,6.111091e-04,0.000072,0.000946,5.398958,0.029596
5,1,1,0,0,1,0,0.333333,-0.666667,5.940882,5.899815,5.943770,5.928156,0.000162,8.031791e-04,0.000244,0.001209,5.743146,0.034228
6,1,0,-1,0,0,1,-0.666667,0.333333,4.944282,4.909488,4.902085,4.918618,0.000659,8.335738e-05,0.000273,0.001015,4.739584,0.032053
7,1,0,1,0,0,1,-0.666667,0.333333,6.172681,6.178504,6.197197,6.182794,0.000102,1.840550e-05,0.000207,0.000328,6.004783,0.031688
8,1,0,0,0,0,0,-0.666667,-0.666667,5.789868,5.798395,5.760068,5.782777,0.000050,2.439237e-04,0.000516,0.000810,5.605259,0.031513


### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Мурат&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Данис ###
## &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;↓&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ↓ ##

![cat](https://www.meme-arsenal.com/memes/27366f67c1b492100085bfebf3415ca2.jpg)

### -Как найти c,m,n? ###
### -Что за t? ###
### -Ты кто? ###